## Obtaining Data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('judge-1377884607_tweet_product_company.csv',encoding='ISO-8859–1')

## Scrubbing/Cleaning Data

In [3]:
data.head(10) #preview of what data looks like

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
6,NaN,NaN,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion


In [16]:
#making corpus
data['tweet_text'].to_list()
#corpus variable for all the test
corpus = data['tweet_text'].to_list()

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jamaalsmith/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
##Tokenize data and then generate FreqDist
from nltk import word_tokenize
tokens = word_tokenize(','.join(corpus))

In [25]:
#FreqDist
from nltk import FreqDist
freq = FreqDist(tokens)
freq.most_common(100)

[('#', 15875),
 (',', 12554),
 ('@', 7194),
 ('mention', 7119),
 ('.', 4957),
 ('SXSW', 4735),
 ('sxsw', 4476),
 ('link', 4311),
 ('}', 4298),
 ('{', 4296),
 ('the', 3926),
 ('to', 3519),
 ('RT', 2945),
 ('at', 2859),
 (';', 2800),
 ('&', 2707),
 ('for', 2439),
 ('!', 2398),
 ('a', 2173),
 ('Google', 2134),
 ('iPad', 2115),
 (':', 2075),
 ('Apple', 1879),
 ('in', 1830),
 ('quot', 1696),
 ('of', 1691),
 ('?', 1659),
 ('is', 1647),
 ('and', 1525),
 ('I', 1460),
 ('iPhone', 1298),
 ('on', 1271),
 ("'s", 1232),
 ('2', 1114),
 ('store', 1046),
 ('-', 972),
 ('you', 944),
 ('Austin', 900),
 ('an', 853),
 ('amp', 836),
 ('with', 804),
 (')', 801),
 ('up', 778),
 ('(', 770),
 ('it', 764),
 ('my', 711),
 ('...', 643),
 ('app', 630),
 ('Circles', 582),
 ('new', 566),
 ('be', 544),
 ('New', 518),
 ('from', 505),
 ('this', 496),
 ('by', 485),
 ('The', 483),
 ("n't", 478),
 ('out', 478),
 ('that', 465),
 ('are', 456),
 ('google', 448),
 ('Android', 445),
 ('your', 431),
 ('not', 426),
 ('Store', 42

In [30]:
#addressing stopwords
from nltk.corpus import stopwords
import string

In [36]:
#stopword treatement continued
#English stopwords
stopwords_list = stopwords.words('english')
#punctuation
stopwords_list+=string.punctuation

In [38]:
#removing stopwords
stopped_tokens = [word.lower() for word in tokens if word.lower() not in stopwords_list]


In [ ]:
#updated FreqDist
freq = F